In [773]:
#This code scrapes dca site to create dataframe of local government authorities
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import html5lib as h5
import bs4
empty = pd.DataFrame()

profile = webdriver.FirefoxProfile()
profile.set_preference("general.useragent.override","Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/40.0.2214.111 Safari/537.36")
driver = webdriver.Firefox(profile)

prelink = 'http://dww2.tceq.texas.gov/DWW/JSP/'
#base = 'http://dww2.tceq.texas.gov/DWW/JSP/SearchDispatch?number=&name=&ActivityStatusCD=All&county=All&WaterSystemType=All&SourceWaterType=All&SampleType=null&begin_date=1%2F01%2F1990&end_date=1%2F01%2F2016&action=Search+For+Water+Systems'
base = 'http://dww2.tceq.texas.gov/DWW/JSP/SearchDispatch?number=&name=&ActivityStatusCD=All&county=BOSQUE&WaterSystemType=All&SourceWaterType=All&SampleType=null&begin_date=2%2F26%2F2014&end_date=2%2F26%2F2016&action=Search+For+Water+Systems'
driver.get(base)
html = driver.page_source
soup = BeautifulSoup(html,"html.parser")
table = soup.findAll("table")
temptable = pd.read_html(str(table[2]),header=0)[0]
linklist = []
for link in wslinks:
    linklist.append(link.get('href'))
temptable['Link'] = linklist

import requests
violationlinks = []
for link in temptable['Link']:
    f = requests.get(prelink+link,timeout=10)
    systempage = BeautifulSoup(f.text,'html.parser')
    systemviolations = systempage.find('a',text = 'Violations').get('href')
    systemviolationlink = prelink + systemviolations
    violationlinks.append(systemviolationlink)
temptable['ViolationPage'] = violationlinks
temptable.to_csv(path_or_buf = '../Input/texas.water.systems.csv')

In [870]:
violdf = pd.DataFrame(columns = ('ViolationNo','CompliancePeriod','TypeCode','ViolationName','AnalyteCode','AnalyteName','ViolationLink'))
for ll in range(0,temptable.shape[0]):
    print(ll)
    f = requests.get(temptable['ViolationPage'][ll])
    violationpage =  BeautifulSoup(f.text,'html.parser')
    temp = violationpage.select('td > font')
    if len(temp) == 0:
        next
    else:
        vnum = 6
        rows = len(temp)/vnum
        store = []
        for i in temp:
            s = i.text
            store.append(s.strip())
        emptydf = pd.DataFrame()
        emptydf = emptydf.append(pd.DataFrame(np.array(store).reshape(rows, vars)))
        emptydf.columns = ('ViolationNo','CompliancePeriod','TypeCode','ViolationName','AnalyteCode','AnalyteName')
        emptydf['Water System No.'] = temptable['Water System No.'][ll]
        vlinks = []
        indivviollinks = violationpage.findAll('a',text='Violation.jsp')
        for i in indivviollinks:
            vlinks.append(i.get('href'))
        emptydf['ViolationLink'] = vlinks
        violdf = violdf.append(emptydf)


0
1
2
3
4
5


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:17: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


ValueError: Length of values does not match length of index

In [881]:
violdf = pd.DataFrame(columns = ('ViolationNo','CompliancePeriod','TypeCode','ViolationName','AnalyteCode','AnalyteName','ViolationLink'))

print(ll)
f = requests.get(temptable['ViolationPage'][5])
violationpage =  BeautifulSoup(f.text,'html.parser')
temp = violationpage.select('td > font')



5


In [890]:
groupviolations = violationpage.findAll('table')[4]
indivviolations = violationpage.findAll('table')[5]

In [922]:
testlink = 'http://dww2.tceq.texas.gov/DWW/JSP/Violations.jsp?tinwsys_is_number=355&tinwsys_st_code=TX&wsnumber=TX0180013%20&DWWState=TX'

In [966]:
import urllib.request
page = urllib.request.urlopen(testlink)
page_text=str(page.read())
new_text=re.sub('</br>','<br>',page_text)
newviolationpage = BeautifulSoup(new_text,"html5lib")

In [969]:

violationpage =  BeautifulSoup(new_text,'html.parser')

In [972]:
tablelist = violationpage.findAll('table')

In [985]:
len(tablelist)

6

In [987]:
tablelist[5]

<table \r\n\t\t\tcellpadding="2" \r\n\t\t\tstyle="border-collapse: collapse" bgcolor="#FFCC99" border="1" bordercolor="#111111" cellspacing="0" width="100%">\r\n\t\t\t<tr>\r\n\t\t\t\t<th bgcolor="#306192" colspan="8"><font \r\n\t\t\t\t\tcolor="#FFFFFF" size="4">Individual Violations</font></th>\r\n\t\t\t</tr>\r\n\t\t\t<tr>\r\n\t\t\t\t<th bgcolor="#306192" width="9%"><span class="style4"><font\r\n\t\t\t\t\tcolor="#ffffff">Violation<br>\r\n\t\t\t\tNo.</br></font\r\n\t\t\t\t\tcolor="#ffffff"></span></th></tr></table>

In [917]:
test2[4]

<table bgcolor="#FFCC99" border="1" bordercolor="#111111" cellpadding="2" cellspacing="0" style="border-collapse: collapse" width="100%">
<tr>
<th bgcolor="#306192" colspan="8"><font color="#FFFFFF" size="4">Individual Violations</font></th>
</tr>
<tr>
<th bgcolor="#306192" width="9%"><span class="style4"><font color="#FFFFFF">Violation<br>
				No.</br></font></span></th>
<th bgcolor="#306192" width="7%"><span class="style4"><font color="#FFFFFF">Compliance Period</font></span></th>
<th bgcolor="#306192" width="10%"><span class="style4"><font color="#FFFFFF">Violation<br>
				Type Code</br></font></span></th>
<th bgcolor="#306192" width="20%"><span class="style4"><font color="#FFFFFF">Violation Name</font></span></th>
<th bgcolor="#306192" width="9%"><span class="style4"><font color="#FFFFFF">Analyte
				Code</font></span></th>
<th bgcolor="#306192" width="19%"><span class="style4"><font color="#FFFFFF">Analyte Name</font></span></th>
<th bgcolor="#306192" width="13%"><span class="styl

In [864]:
    if len(temp) == 0:
        next
    else:
        vnum = 6
        rows = len(temp)/vnum
        store = []
        for i in temp:
            s = i.text
            store.append(s.strip())
        emptydf = pd.DataFrame()
        emptydf = emptydf.append(pd.DataFrame(np.array(store).reshape(rows, vars)))
        emptydf.columns = ('ViolationNo','CompliancePeriod','TypeCode','ViolationName','AnalyteCode','AnalyteName')
        emptydf['Water System No.'] = temptable['Water System No.'][ll]
        vlinks = []
        indivviollinks = violationpage.findAll('a',text='Violation.jsp')
        for i in indivviollinks:
            vlinks.append(i.get('href'))
        emptydf['ViolationLink'] = vlinks
        violdf = violdf.append(emptydf)

,ViolationNo,CompliancePeriod,TypeCode,ViolationName,AnalyteCode,AnalyteName,Water System No.
0,2014-10,07-01-2013-,51,INITIAL TAP SAMPLING (LCR),5000,LEAD & COPPER RULE,TX0180085
1,2013-7,01-01-2013-,51,INITIAL TAP SAMPLING (LCR),5000,LEAD & COPPER RULE,TX0180085
2,2013-4,07-01-2012-,51,INITIAL TAP SAMPLING (LCR),5000,LEAD & COPPER RULE,TX0180085
3,2012-1,05-01-2011-,23,"MONITORING (TCR), ROUTINE MAJOR",3100,COLIFORM (TCR),TX0180085


In [825]:
    vlinks = []
    for vl in linkstoviolations:
        vlinks.append(prelink + l['href'])

,AnalyteCode,AnalyteName,CompliancePeriod,TypeCode,ViolationName,ViolationNo,Water System No.
0,2950,TTHM,10-01-2014-,02,"MCL, LRAA",2015-722,TX0180001
1,2950,TTHM,07-01-2014-,02,"MCL, LRAA",2014-720,TX0180001
2,0200,SWTR,02-01-2003-,36,"MONITORING, RTN/RPT MAJOR (SWTR-FILTER)",2003-603,TX0180001
3,0200,SWTR,01-01-2003-,36,"MONITORING, RTN/RPT MAJOR (SWTR-FILTER)",2003-503,TX0180001
4,0200,SWTR,12-01-2002-,36,"MONITORING, RTN/RPT MAJOR (SWTR-FILTER)",2003-103,TX0180001
5,0200,SWTR,11-01-2002-,36,"MONITORING, RTN/RPT MAJOR (SWTR-FILTER)",2003-203,TX0180001
6,0200,SWTR,10-01-2002-,36,"MONITORING, RTN/RPT MAJOR (SWTR-FILTER)",2003-403,TX0180001
7,0200,SWTR,09-01-2002-,36,"MONITORING, RTN/RPT MAJOR (SWTR-FILTER)",2002-302,TX0180001
0,0999,CHLORINE,07-01-2015-,27,"MONITORING, ROUTINE (DBP), MAJOR",2015-100068844,TX0180002
1,7500,PUBLIC NOTICE,07-01-2014-,75,PUBLIC NOTICE RULE LINKED TO VIOLATION,2015-100068843,TX0180002


In [809]:

    emptydf['Link'] = vlinks
    emptydf['Water System No.'] = temptable['Water System No.'][ll]

'http://dww2.tceq.texas.gov/DWW/JSP/Violations.jsp?tinwsys_is_number=351&tinwsys_st_code=TX&wsnumber=TX0180001   &DWWState=TX'

In [807]:
temptable

,Water System No.,Water System Name,Type,Status,Pri. Cnty Served,Pri. Src. Water Type,Link,ViolationPage
0,TX0180001,CITY OF CLIFTON Fact Sheet Summary Sheet,C,A,BOSQUE,SW,WaterSystemDetail.jsp?tinwsys_is_number=351&ti...,http://dww2.tceq.texas.gov/DWW/JSP/Violations....
1,TX0180002,CITY OF MERIDIAN Fact Sheet Summary Sheet,C,A,BOSQUE,SWP,WaterSystemDetail.jsp?tinwsys_is_number=352&ti...,http://dww2.tceq.texas.gov/DWW/JSP/Violations....
2,TX0180003,CITY OF VALLEY MILLS Fact Sheet Summary Sheet,C,A,BOSQUE,GW,WaterSystemDetail.jsp?tinwsys_is_number=353&ti...,http://dww2.tceq.texas.gov/DWW/JSP/Violations....
3,TX0180004,USCOE WALLING BEND PARK Fact Sheet Summary S...,NP,I,BOSQUE,NaN,WaterSystemDetail.jsp?tinwsys_is_number=7216&t...,http://dww2.tceq.texas.gov/DWW/JSP/Violations....
4,TX0180009,BOSQUE BRAZOS VALLEY WATER SYSTEM Fact Sheet ...,NP,I,BOSQUE,NaN,WaterSystemDetail.jsp?tinwsys_is_number=354&ti...,http://dww2.tceq.texas.gov/DWW/JSP/Violations....
5,TX0180013,CITY OF CRANFILLS GAP Fact Sheet Summary Sheet,C,A,BOSQUE,GW,WaterSystemDetail.jsp?tinwsys_is_number=355&ti...,http://dww2.tceq.texas.gov/DWW/JSP/Violations....
6,TX0180014,CITY OF IREDELL Fact Sheet Summary Sheet,C,A,BOSQUE,GW,WaterSystemDetail.jsp?tinwsys_is_number=356&ti...,http://dww2.tceq.texas.gov/DWW/JSP/Violations....
7,TX0180015,ROCKY CREEK WATER SYSTEM Fact Sheet Summary ...,C,A,BOSQUE,GW,WaterSystemDetail.jsp?tinwsys_is_number=357&ti...,http://dww2.tceq.texas.gov/DWW/JSP/Violations....
8,TX0180016,LAKESIDE WATER SUPPLY DISTRICT Fact Sheet Su...,C,A,BOSQUE,GW,WaterSystemDetail.jsp?tinwsys_is_number=358&ti...,http://dww2.tceq.texas.gov/DWW/JSP/Violations....
9,TX0180018,CITY OF MORGAN Fact Sheet Summary Sheet,C,A,BOSQUE,GW,WaterSystemDetail.jsp?tinwsys_is_number=359&ti...,http://dww2.tceq.texas.gov/DWW/JSP/Violations....


In [607]:
link = 'http://dww2.tceq.texas.gov/DWW/JSP/Violation.jsp?tinwsys_is_number=351&tinwsys_st_code=TX&wsnumber=TX0180001%20%20%20&DWWState=TX&tmnviol_is_number=211711&tmnviol_st_code=TX'
f = requests.get(link)
violationpage =  BeautifulSoup(f.text,'html.parser')

In [573]:
tables = violationpage.find('body').find_all('table', recursive=False)
#print(tables[1].text.strip())

In [714]:
test = violationpage.findAll('a',href=TRUE,text = 'Violation')

In [716]:
systempage.find('a',text = 'Violations').get('href')

[]

In [707]:
emptyviolationdf = pd.DataFrame(columns = ('Violation  No:', 'Violation  Type:', 'Violation  Category:',
       'Analyte  code', 'Comp  Prd Begin Date', 'Determination  Date:',
       'Violation  Name:', 'Status', 'Analyte  Name', 'Comp  Prd End Date'))
temp = pd.read_html(str(test[4]),header=0)[0]
t1 = temp.ix[:,:2]
t2 = temp.ix[:,2:4]
t2.columns = t1.columns =  ['Var','Value']
t3 = t1.append(t2)
varnames = t3['Var']
t4 = t3.T[1:]
t4.columns = varnames
emptyviolationdf = emptyviolationdf.append(t4)

temp = pd.read_html(str(test[5]),header=1)[0]
temp['Violation'] = t4['Violation  No:'][0]
emptyenforcmentdf = pd.DataFrame(columns = ('EA  Number', 'Action  Type',
       'PN Due/Proof PN Due  PN Done/Proof PN Done', 'Action  Name', 'Status',
       'Date', 'Violation'))

Index(['EA  Number', 'Action  Type',
       'PN Due/Proof PN Due  PN Done/Proof PN Done', 'Action  Name', 'Status',
       'Date', 'Violation'],
      dtype='object')

,Violation No:,Violation Type:,Violation Category:,Analyte code,Comp Prd Begin Date,Determination Date:,Violation Name:,Status,Analyte Name,Comp Prd End Date
